In [1]:
import tensorflow as tf
import numpy as np

In [1]:
BATCH_SIZE = 64

# Set hyper parameters of neural network
# Define the feature size and number of channels that the neural network can receive
FEATURES_SIZE = 9
NUM_CHANNELS = 1

# Define the size and number of first-level convolution kernels
CONV1_SIZE = 5
CONV1_KERNEL_NUM = 32

# Define the size and number of second-level convolution kernels
CONV2_SIZE = 5
CONV2_KERNEL_NUM = 64

# Define the number of neurons in the third fully connected layer
FC_SIZE = 512

# Define the number of neurons in the fourth fully connected layer
OUTPUT_NODE = 2

In [2]:
def inference(features,regularizer,keep_prob):
    """Build the CIFAR-10 model
    conv1-->pooling1-->conv2-->pooling_2-->local3-->local4
    @param images: feed data[batch_size, height, width, 1][64,9,9,3]
    @return: logits: Prediction vector
    """
    x_features = tf.reshape(features, [-1, FEATURES_SIZE,FEATURES_SIZE, 1])

    ## convl layer ##
    W_conv1 = weight_variable([CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_KERNEL_NUM],regularizer) # kernel 5*5, channel is 1, out size 32，regularizer is the regularization parameter。
    b_conv1 = bias_variable([CONV1_KERNEL_NUM])
    h_conv1 = tf.nn.relu(conv2d(x_features,W_conv1) + b_conv1)  # output size 9*9*32
    h_pool1 = max_pool_2x2(h_conv1)                          

    ## conv2 layer ##
    W_conv2 =weight_variable([CONV2_SIZE,CONV2_SIZE,CONV1_KERNEL_NUM, CONV2_KERNEL_NUM],regularizer) # kernel 5*5, in size 32, out size 64
    b_conv2 =bias_variable([CONV2_KERNEL_NUM])
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)  # output size 9*9*64
    h_pool2 = max_pool_2x2(h_conv2)                          

    
    pool_shape = h_pool2.get_shape().as_list()
    
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    # reshaped = tf.reshape(h_pool2, [pool_shape[0], nodes])
    reshaped = tf.layers.flatten(h_pool2)
    # Initialize the weight of the fully connected layer and add regularization
    fc1_w = weight_variable([nodes, FC_SIZE], regularizer)
    # Initialize the bias item of the fully connected layer
    fc1_b = bias_variable([FC_SIZE])
    

    # Do matrix multiplication of the converted reshaped vector and weight fc1_w, then add the offset, and finally use relu to activate
    fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_w) + fc1_b)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    # If it is in the training phase, use dropout on the output of this layer, that is, randomly disable half of the neurons in the output of this layer. It is set to avoid overfitting. It is generally only used in the fully connected layer.
    # Implement the forward propagation process of the fourth fully connected layer,and initialize the variables corresponding to the fully connected layer.
    
    fc2_w = weight_variable([FC_SIZE, OUTPUT_NODE], regularizer)
    fc2_b = bias_variable([OUTPUT_NODE])
    t_logits = tf.matmul(fc1, fc2_w) + fc2_b
    return t_logits

In [3]:
def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs:v_xs, keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

In [4]:
def weight_variable(shape,regularizer):
    initial = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    if regularizer != None: 
        tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(initial)) 
    return initial

In [5]:
def bias_variable(shape):
   # initial = tf.constant(0.1, shape=shape)
    b = tf.Variable(tf.zeros(shape))
    return  b

In [6]:
def conv2d(x, W):
    # stride[1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] =1
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")  # padding="SAME"用零填充边界

In [7]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,1,1,1], strides=[1,1,1,1], padding="SAME")

In [10]:
def accuracy(logits, labels):
    reshape_logits = tf.cast(tf.argmax(logits, 1), tf.int32)
    accuracy_value = tf.reduce_mean(tf.cast(tf.equal(reshape_logits, labels), dtype=tf.float32))
    num=tf.cast(tf.equal(reshape_logits, labels), dtype=tf.float32)
    return accuracy_value,num,labels,reshape_logits

In [11]:
def Epoch_accuracy_mean(accuracy):
    Epoch_accuracy = np.mean(accuracy)
    return Epoch_accuracy

In [12]:
def count_nums(true_labels, num_classes):
    initial_value = 0
    list_length = num_classes
    list_data = [ initial_value for i in range(list_length)]
    for i in range(0, num_classes):
        list_data[i] = true_labels.count(i)
    return list_data

In [13]:
def accuracyofeachclass(confusion_matrix, true_labels, num_classes):
    # Number of test samples in each category
    list_data = count_nums(true_labels, num_classes)
    # Number of correct classifications in each category
    initial_value = 0
    list_length = num_classes
    true_pred = [ initial_value for i in range(list_length)]
    for i in range(0,num_classes):
        true_pred[i] = confusion_matrix[i][i]
 
    # Calculate the correct rate of each sample being correctly classified
    acc = []
    for i in range(0, num_classes):
        acc.append(0)
 
    for i in range(0,num_classes):
        acc[i] = true_pred[i] / list_data[i]
 
    return acc